In [1]:
!pip install websockets

In [2]:
!pip install flask flask-socketio eventlet

In [3]:
from flask import Flask, render_template, request
from flask_socketio import SocketIO, send
import sqlite3
from threading import Thread
import nest_asyncio

nest_asyncio.apply()

In [4]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

In [5]:
def init_db():
    conn = sqlite3.connect('chat.db')
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS messages (
        id INTEGER PRIMARY KEY,
        username TEXT NOT NULL,
        content TEXT NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    conn.commit()
    conn.close()

init_db()


In [6]:
@app.route('/')
def index():
    return '''
    <!doctype html>
    <html>
    <head>
        <title>Chat Application</title>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/3.1.3/socket.io.js"></script>
        <script type="text/javascript">
            document.addEventListener("DOMContentLoaded", function() {
                var socket = io();
                socket.on('message', function(msg) {
                    var messages = document.getElementById('messages');
                    var message = document.createElement('div');
                    message.innerText = msg.username + ': ' + msg.content;
                    messages.appendChild(message);
                });

                document.getElementById('sendBtn').onclick = function() {
                    var username = document.getElementById('username').value;
                    var content = document.getElementById('message').value;
                    socket.send({username: username, content: content});
                };
            });
        </script>
    </head>
    <body>
        <h1>Chat Room</h1>
        <div id="messages"></div>
        <input type="text" id="username" placeholder="Username">
        <input type="text" id="message" placeholder="Message">
        <button id="sendBtn">Send</button>
    </body>
    </html>
    '''

@socketio.on('message')
def handle_message(msg):
    username = msg['username']
    content = msg['content']
    
    conn = sqlite3.connect('chat.db')
    c = conn.cursor()
    c.execute("INSERT INTO messages (username, content) VALUES (?, ?)", (username, content))
    conn.commit()
    conn.close()
    
    send(msg, broadcast=True)


In [8]:
def run_app():
    socketio.run(app, host='0.0.0.0', port=5000)

thread = Thread(target=run_app)
thread.start()

Exception in thread Thread-10 (run_app):
Traceback (most recent call last):
  File "C:\Users\KIIT\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\KIIT\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Local\Temp\ipykernel_2888\3179399158.py", line 2, in run_app
  File "C:\Users\KIIT\anaconda3\Lib\site-packages\flask_socketio\__init__.py", line 684, in run
    run_server()
  File "C:\Users\KIIT\anaconda3\Lib\site-packages\flask_socketio\__init__.py", line 661, in run_server
    eventlet_socket = eventlet.listen(addresses[0][4],
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\KIIT\anaconda3\Lib\site-packages\eventlet\convenience.py", line 78, in listen
    sock.bind(addr)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted
